# Data Augmentation on Sample from Dataset

### Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from io import StringIO
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader

### Load Data

In [ ]:
# Transformations to apply to inputs
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.2, inplace=True)])